In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_442915/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 14:26:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 14:26:19 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [34]:
import pandas as pd

# Sample data for Products table
products_data = {
    'product_id': [1, 2, 3],
    'LC_Store': [100, None, None],
    'Nozama': [None, 200, None],
    'Shop': [110, None, 1000],
    'Souq': [None, 190, 1900]
}

# Create DataFrame for Products
products_df = pd.DataFrame(products_data)

# Display the DataFrame
print("Products table:")
print(products_df)



df_product = spark.createDataFrame(products_df)
df_product.createOrReplaceTempView("Products")



Products table:
   product_id  LC_Store  Nozama    Shop    Souq
0           1     100.0     NaN   110.0     NaN
1           2       NaN   200.0     NaN   190.0
2           3       NaN     NaN  1000.0  1900.0


In [35]:
store_names = df_product.columns
store_names.remove("product_id")
store_names

['LC_Store', 'Nozama', 'Shop', 'Souq']

In [36]:
query = "\nunion all\n".join([f"select product_id, '{store_name}' as store, {store_name} as price from Products" for store_name in store_names])

In [37]:
print(query)

select product_id, 'LC_Store' as store, LC_Store as price from Products
union all
select product_id, 'Nozama' as store, Nozama as price from Products
union all
select product_id, 'Shop' as store, Shop as price from Products
union all
select product_id, 'Souq' as store, Souq as price from Products


In [38]:
query_1 = f"""
    select
        *
    from (
        {query}
    ) where price != 'NULL'
"""

In [41]:
spark.sql(query).printSchema()
spark.sql(query).show()

root
 |-- product_id: long (nullable = true)
 |-- store: string (nullable = false)
 |-- price: double (nullable = true)

+----------+--------+------+
|product_id|   store| price|
+----------+--------+------+
|         1|LC_Store| 100.0|
|         2|LC_Store|   NaN|
|         3|LC_Store|   NaN|
|         1|  Nozama|   NaN|
|         2|  Nozama| 200.0|
|         3|  Nozama|   NaN|
|         1|    Shop| 110.0|
|         2|    Shop|   NaN|
|         3|    Shop|1000.0|
|         1|    Souq|   NaN|
|         2|    Souq| 190.0|
|         3|    Souq|1900.0|
+----------+--------+------+

